In [1]:
import pandas as pd
import numpy as np
from pulp import *
from scipy.optimize import linprog

In [2]:
#import data into df
#start with hub and city capacity
hub_capacity = pd.read_excel('Amazon_data.xlsx', sheet_name='Hub_Capacity')
print(hub_capacity)

  City  Current tons  Capacity
0  CVG         82800     95650
1  AFW         38400     44350


In [3]:
#focus_capcity
focus_capacity = pd.read_excel('Amazon_data.xlsx', sheet_name='Focus_Capacity')
print(focus_capacity)

             City  Capacity
0         Leipzig     85000
1       Hyderabad     19000
2  San Bernardino     36000


In [4]:
#demand
demand = pd.read_excel('Amazon_data.xlsx', sheet_name='Demand')
print(demand)

              City  Demand
0            Paris    6500
1          Cologne     640
2          Hanover     180
3       Bangalore     9100
4       Coimbatore     570
..             ...     ...
60         Houston    3300
61     San Antonio    1100
62        Richmond     600
63  Seattle/Tacoma    2000
64         Spokane     260

[65 rows x 2 columns]


In [5]:
#focus_cost
focus_cost = pd.read_excel('Amazon_data.xlsx', sheet_name='Focus_Cost')
#fill null values
focus_cost=focus_cost.replace(np.nan, '3')
print(focus_cost)

             City  CVG  AFW Leipzig Hyderabad San Bernardino
0         Leipzig  1.5    3       3         3              3
1       Hyderabad    3    3     1.6         3              3
2  San Bernardino  0.5  0.5       3         3              3


In [6]:
#Center_cost
center_cost = pd.read_excel('Amazon_data.xlsx', sheet_name='Center_Cost')
#fill null values
center_cost=center_cost.replace(np.nan, '3')
print(center_cost)

              City  CVG   AFW Leipzig Hyderabad San Bernardino
0            Paris  1.6     3     0.5       1.1              3
1          Cologne  1.5     3     0.5       1.0              3
2          Hanover  1.5     3     0.5       1.0              3
3        Bengaluru    3     3     1.5       0.5              3
4       Coimbatore    3     3     1.5       0.5              3
..             ...  ...   ...     ...       ...            ...
60         Houston  0.5  0.25       3         3            0.5
61     San Antonio  0.5  0.25       3         3            0.5
62        Richmond  0.5   0.5       3         3            0.7
63  Seattle/Tacoma  0.5   0.5       3         3            0.5
64         Spokane  0.5   0.5       3         3            0.5

[65 rows x 6 columns]


In [7]:
#cost to numeric
focus_cost.iloc[:, 1:] = focus_cost.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
center_cost.iloc[:, 1:] = center_cost.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

In [8]:
#mapping dictionary to standardize city names
city_mapping = {
    'bangalore': 'bengaluru',
    'san juan': 'san juan',
    'chicago/rockford': 'chicago',
    'dallas/fort worth': 'dallas',
}
# Convert city names to lowercase
demand['City'] = demand['City'].str.strip().str.lower()
center_cost['City'] = center_cost['City'].str.strip().str.lower()
#apply mapping
demand['City'] = demand['City'].apply(lambda x: city_mapping.get(x, x))
demand = demand[demand['City'] != 'rockford']
center_cost['City'] = center_cost['City'].apply(lambda x: city_mapping.get(x, x))


# Verify if the names are consistent
print("Demand cities:", demand['City'].unique())
print("Center cost cities:", center_cost['City'].unique())

Demand cities: ['paris' 'cologne' 'hanover' 'bengaluru' 'coimbatore' 'delhi' 'mumbai'
 'cagliari' 'catania' 'milan' 'rome' 'katowice' 'barcelona' 'madrid'
 'castle donington' 'london' 'mobile' 'anchorage' 'fairbanks' 'phoenix'
 'los angeles' 'ontario' 'riverside' 'sacramento' 'san francisco'
 'stockton' 'denver' 'hartford' 'miami' 'lakeland' 'tampa' 'atlanta'
 'honolulu' 'kahului/maui' 'kona' 'chicago' 'fort wayne' 'south bend'
 'des moines' 'wichita' 'new orleans' 'baltimore' 'minneapolis'
 'kansas city' 'st. louis' 'omaha' 'manchester' 'albuquerque' 'new york'
 'charlotte' 'toledo' 'wilmington' 'portland' 'allentown' 'pittsburgh'
 'san juan' 'nashville' 'austin' 'dallas' 'houston' 'san antonio'
 'richmond' 'seattle/tacoma' 'spokane']
Center cost cities: ['paris' 'cologne' 'hanover' 'bengaluru' 'coimbatore' 'delhi' 'mumbai'
 'cagliari' 'catania' 'milan' 'rome' 'katowice' 'barcelona' 'madrid'
 'castle donington' 'london' 'mobile' 'anchorage' 'fairbanks' 'phoenix'
 'los angeles' 'ontari

In [9]:
#check if every city in demand exists in center_cost
missing_in_center_cost = []
for city in demand['City']:
    if city not in center_cost['City'].values:
        missing_in_center_cost.append(city)

if missing_in_center_cost:
    print("Cities in demand not found in center_cost:", missing_in_center_cost)
else:
    print("All cities from demand are found in center_cost.")

All cities from demand are found in center_cost.


In [10]:
#Define the problem
prob = LpProblem("Amazon_Distribution", LpMinimize)

In [11]:
#create dictionaries to store decision variables
x = {}  #shipment from hub to focus city
y = {}  #shipment from hub to center
z = {}  #shipment from focus city to center

In [12]:
#define x_ij (hub to focus city)
for i in hub_capacity['City']:
    for j in focus_capacity['City']:
        x[(i, j)] = LpVariable(f"x_{i}_{j}", lowBound=0, cat='Continuous')

#Define y_ik (hub to center)
for i in hub_capacity['City']:
    for k in demand['City']:
        y[(i, k)] = LpVariable(f"y_{i}_{k}", lowBound=0, cat='Continuous')

#Define z_jk (focus city to center)
for j in focus_capacity['City']:
    for k in demand['City']:
        z[(j, k)] = LpVariable(f"z_{j}_{k}", lowBound=0, cat='Continuous')

In [13]:
#Objective Function
objective = 0

In [14]:
# Costs for x_ij (hub to focus city)
for i in hub_capacity['City']:
    for j in focus_capacity['City']:
        cost = focus_cost.loc[focus_cost['City'] == j, i].values[0]
        if isinstance(cost, (int, float)):
            objective += cost * x[(i, j)]

# Costs for y_ik (hub to center)
for i in hub_capacity['City']:
    for k in demand['City']:
        cost = center_cost.loc[center_cost['City'] == k, i].values[0]
        if isinstance(cost, (int, float)):
            objective += cost * y[(i, k)]

# Costs for z_jk (focus city to center)
for j in focus_capacity['City']:
    for k in demand['City']:
        cost = center_cost.loc[center_cost['City'] == k, j].values[0]
        if isinstance(cost, (int, float)):
            objective += cost * z[(j, k)]

In [15]:
#minimize cost
prob += objective

In [16]:
# Print the objective function expression
print(f"Objective Function (linear expression): {prob.objective}")

Objective Function (linear expression): 3.0*x_AFW_Hyderabad + 3.0*x_AFW_Leipzig + 0.5*x_AFW_San_Bernardino + 3.0*x_CVG_Hyderabad + 1.5*x_CVG_Leipzig + 0.5*x_CVG_San_Bernardino + 0.5*y_AFW_albuquerque + 0.5*y_AFW_allentown + y_AFW_anchorage + 0.5*y_AFW_atlanta + 0.25*y_AFW_austin + 0.5*y_AFW_baltimore + 3.0*y_AFW_barcelona + 3.0*y_AFW_bengaluru + 3.0*y_AFW_cagliari + 3.0*y_AFW_castle_donington + 3.0*y_AFW_catania + 0.5*y_AFW_charlotte + 0.5*y_AFW_chicago + 3.0*y_AFW_coimbatore + 3.0*y_AFW_cologne + 3.0*y_AFW_dallas + 3.0*y_AFW_delhi + 0.5*y_AFW_denver + 0.5*y_AFW_des_moines + y_AFW_fairbanks + 0.5*y_AFW_fort_wayne + 3.0*y_AFW_hanover + 0.5*y_AFW_hartford + 0.5*y_AFW_honolulu + 0.25*y_AFW_houston + 0.5*y_AFW_kahului_maui + 0.5*y_AFW_kansas_city + 3.0*y_AFW_katowice + 0.5*y_AFW_kona + 0.5*y_AFW_lakeland + 3.0*y_AFW_london + 0.5*y_AFW_los_angeles + 3.0*y_AFW_madrid + 0.5*y_AFW_manchester + 0.5*y_AFW_miami + 3.0*y_AFW_milan + 0.5*y_AFW_minneapolis + 0.5*y_AFW_mobile + 3.0*y_AFW_mumbai + 0.5

In [17]:
#ADD CONSTRAINTS
# Hub capacity constraint: The total shipment from each hub should not exceed its capacity
for i in hub_capacity['City']:
    prob += lpSum(x[(i, j)] for j in focus_capacity['City']) + lpSum(y[(i, k)] for k in demand['City']) <= hub_capacity.loc[hub_capacity['City'] == i, 'Capacity'].values[0], f"Hub_{i}_Capacity_Constraint"

# Focus city capacity constraint: The total shipment into each focus city should not exceed its capacity
for j in focus_capacity['City']:
    prob += lpSum(x[(i, j)] for i in hub_capacity['City']) <= focus_capacity.loc[focus_capacity['City'] == j, 'Capacity'].values[0], f"Focus_City_{j}_Capacity_Constraint"

# Flow balance for focus cities: Inflow (x_ij) = Outflow (z_jk)
for j in focus_capacity['City']:
    prob += lpSum(x[(i, j)] for i in hub_capacity['City']) == lpSum(z[(j, k)] for k in demand['City']), f"Focus_City_{j}_Flow_Balance"

# Center demand constraint: The total shipments into a center (from hubs and focus cities) should meet the demand
for k in demand['City']:
    prob += lpSum(y[(i, k)] for i in hub_capacity['City']) + lpSum(z[(j, k)] for j in focus_capacity['City']) == demand.loc[demand['City'] == k, 'Demand'].values[0], f"Center_{k}_Demand_Constraint"

print("List of Constraints:")
for constraint_name, constraint in prob.constraints.items():
    print(f"{constraint_name}: {constraint}")

List of Constraints:
Hub_CVG_Capacity_Constraint: x_CVG_Hyderabad + x_CVG_Leipzig + x_CVG_San_Bernardino + y_CVG_albuquerque + y_CVG_allentown + y_CVG_anchorage + y_CVG_atlanta + y_CVG_austin + y_CVG_baltimore + y_CVG_barcelona + y_CVG_bengaluru + y_CVG_cagliari + y_CVG_castle_donington + y_CVG_catania + y_CVG_charlotte + y_CVG_chicago + y_CVG_coimbatore + y_CVG_cologne + y_CVG_dallas + y_CVG_delhi + y_CVG_denver + y_CVG_des_moines + y_CVG_fairbanks + y_CVG_fort_wayne + y_CVG_hanover + y_CVG_hartford + y_CVG_honolulu + y_CVG_houston + y_CVG_kahului_maui + y_CVG_kansas_city + y_CVG_katowice + y_CVG_kona + y_CVG_lakeland + y_CVG_london + y_CVG_los_angeles + y_CVG_madrid + y_CVG_manchester + y_CVG_miami + y_CVG_milan + y_CVG_minneapolis + y_CVG_mobile + y_CVG_mumbai + y_CVG_nashville + y_CVG_new_orleans + y_CVG_new_york + y_CVG_omaha + y_CVG_ontario + y_CVG_paris + y_CVG_phoenix + y_CVG_pittsburgh + y_CVG_portland + y_CVG_richmond + y_CVG_riverside + y_CVG_rome + y_CVG_sacramento + y_CVG_

In [18]:
print(f"Cost for hub {i}, focus city {j}: {cost}")
print(f"Hub capacity constraint for {i}: {hub_capacity.loc[hub_capacity['City'] == i, 'Capacity'].values[0]}")

Cost for hub AFW, focus city San Bernardino: 0.5
Hub capacity constraint for AFW: 44350


In [19]:
#Solve the LP problem
prob.solve()

if LpStatus[prob.status] == "Optimal":
    print("Optimal Solution Found")
    print(f"Minimum Distribution Cost: {value(prob.objective)}")
else:
    print("No Optimal Solution Found")

Optimal Solution Found
Minimum Distribution Cost: 199390.25


In [20]:
if LpStatus[prob.status] == "Optimal":
    print("\nHub Capacity Usage :")
    for i in hub_capacity['City']:
        total_used_capacity = sum(v.varValue for v in prob.variables() if v.name.startswith(f"x_{i}_") or v.name.startswith(f"y_{i}_"))
        if total_used_capacity > 0:
            print(f"{i} used capacity: {total_used_capacity}")
        else:
            print(f"{i}No capacity used")
    
    print("\nFocus City Capacity Usage:")
    for j in focus_capacity['City']:
        total_used_capacity = sum(v.varValue for v in prob.variables() if v.name.startswith(f"x_{i}_{j}"))
        if total_used_capacity > 0:
            print(f"{j} used capacity: {total_used_capacity}")
        else:
            print(f"{j} No capacity used")


Hub Capacity Usage :
CVG used capacity: 95650.0
AFW used capacity: 37925.0

Focus City Capacity Usage:
Leipzig No capacity used
Hyderabad No capacity used
San Bernardino No capacity used


In [22]:
for name, constraint in prob.constraints.items():
    print(f"{name}: Slack = {constraint.slack}")

Hub_CVG_Capacity_Constraint: Slack = -0.0
Hub_AFW_Capacity_Constraint: Slack = 6425.0
Focus_City_Leipzig_Capacity_Constraint: Slack = 85000.0
Focus_City_Hyderabad_Capacity_Constraint: Slack = 19000.0
Focus_City_San_Bernardino_Capacity_Constraint: Slack = 36000.0
Focus_City_Leipzig_Flow_Balance: Slack = -0.0
Focus_City_Hyderabad_Flow_Balance: Slack = -0.0
Focus_City_San_Bernardino_Flow_Balance: Slack = -0.0
Center_paris_Demand_Constraint: Slack = -0.0
Center_cologne_Demand_Constraint: Slack = -0.0
Center_hanover_Demand_Constraint: Slack = -0.0
Center_bengaluru_Demand_Constraint: Slack = -0.0
Center_coimbatore_Demand_Constraint: Slack = -0.0
Center_delhi_Demand_Constraint: Slack = -0.0
Center_mumbai_Demand_Constraint: Slack = -0.0
Center_cagliari_Demand_Constraint: Slack = -0.0
Center_catania_Demand_Constraint: Slack = -0.0
Center_milan_Demand_Constraint: Slack = -0.0
Center_rome_Demand_Constraint: Slack = -0.0
Center_katowice_Demand_Constraint: Slack = -0.0
Center_barcelona_Demand_Const